# 1 Импорт библиотек

In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pickle

# 2 Загрузка данных

In [94]:
file_path = '../data/tickets.csv'

data = pd.read_csv(file_path, delimiter=';')

In [ ]:
# Проверка корректной загрузки данных
print("Пример данных:")
display(data.head())

# 3 Предобработка данных

In [96]:
# Удаляем строки с пропущенными значениями
data = data.dropna(subset=['Description', 'Type'])

In [97]:
# Приведение текста к нижнему регистру
data['Description'] = data['Description'].str.lower()

In [98]:
# Удаление лишних символов (шум)
data['Description'] = data['Description'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [ ]:
# Выводим несколько обработанных записей
print("\nОбработанные данные:")
display(data.head())

# 3 Разделение на признаки и целевые переменные, векторизация данных TF-IDF

In [100]:
X = data['Description']  # Признаки
y = data['Type']         # Целевая переменная

## Создание обучающей и валидационной выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nКоличество записей в обучающей выборке: {len(X_train)}")
print(f"Количество записей в валидационной выборке: {len(X_test)}")

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("\nРазмерность векторизованных данных:")
print(f"Обучающие данные: {X_train_tfidf.shape}")
print(f"Валидационные данные: {X_test_tfidf.shape}")

# 4 Обучение модели с использованием метода логистической регрессии

In [ ]:
model = LogisticRegression(random_state=42, max_iter=1000)

model.fit(X_train_tfidf, y_train)

# 5 Проверка работоспособности и оценка модели

In [ ]:
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)


print("\nAccuracy модели:", accuracy)
print("\nОтчет по классификации:\n", classification_rep)

# 6 Сохранение модели

In [105]:
# Директория для сохранения модели
model_path = '../models/ticket_classifier.pkl'
vectorizer_path = '../models/tfidf_vectorizer.pkl'

# Сохранение модели
with open(model_path, 'wb') as model_file:
    pickle.dump(model, model_file)

# Сохранение векторизатора
with open(vectorizer_path, 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

# 7 Загрузка и проверка модели

In [ ]:
# Загрузка модели
with open(model_path, 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Загрузка векторизатора
with open(vectorizer_path, 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

print("\nМодель и векторизатор успешно загружены.")

In [ ]:
# Тестирование загруженной модели на новом тексте
new_ticket = ["Kittens are very cute creatures"]
new_ticket_vectorized = loaded_vectorizer.transform(new_ticket)
new_ticket_prediction = loaded_model.predict(new_ticket_vectorized)

print("\nТестовое описание тикета:", new_ticket[0])
print("Предсказанная категория:", new_ticket_prediction[0])